<a href="https://colab.research.google.com/github/moustafa-7/YOLOv3/blob/master/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
#os.mkdir("cfg")
os.chdir("cfg")

In [4]:
# download the configuration of the yolov3 network
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

--2019-06-27 16:10:01--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2019-06-27 16:10:01 (103 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



In [0]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [0]:
def parse_cfg(cfgfile):



In [15]:
!ls

yolov3.cfg


In [0]:
def parse_cfg(cfgfile):
  block = {}
  blocks = []

  for line in lines:
      if line[0] == "[":               # This marks the start of a new block
          if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
              blocks.append(block)     # add it the blocks list
              block = {}               # re-init the block
          block["type"] = line[1:-1].rstrip()     
      else:
          key,value = line.split("=") 
          block[key.rstrip()] = value.lstrip()
  blocks.append(block)

  return blocks
  

In [0]:
def create_modules(blocks):
  net_info  = blocks[0]
  module_list = nn.ModuleList()
  prev_filters = 3
  output_filter = []
  
  for index, x in enumerate(blocks[1:]):
        module = nn.Sequential()
        
        if x["type"] == "convolutional":
          activation = x["activation"]
